# Data Exploration

In [1]:
# Initialize Spark, RERUN EVERY TIME!
from pyspark import SparkConf, SparkContext


conf = SparkConf()\
        .setMaster("local")\
        .setAppName("Sandbox")

sc = SparkContext(conf = conf)

In [2]:
# Index Constants
TRANS_ALLOC = "cast-allocation-ornl"
TRANS_STEP  = "cast-allocation-step-ornl"
STEP_HIST   = "cast-db-csm_step_history-ornl-*"
NODE_HIST   = "cast-db-csm_allocation_node_history-ornl-*"
ALLOC_HIST  = "cast-db-csm_allocation_history-ornl-*"

# Set the Connection Constants
NODES       = "10.7.4.15,10.7.4.17"
PORTS       = "9200"

# Defining Node  History

We want to get some data from the allocation node history table before anything else and leverage Elasticsearch. Step 1 is to determine which fields we want and how we're going to aggregate them (tons of data is great, but skew can be just as if not more interesting) Using this we can start to refine the allocation data to make it easier to run analytics on the allocation data. The following variables are all in the **data** object. The data will be condensed to pivot on the **data.allocation_id**. 

**NOTE:** This is one way of analyzing the data. Other analytics may be written using the raw allocation node data, but I'm interested in a few specific indicators to apply to the overall allocation.

* **memory_usage_max**
    * Should store the Average and the Maximum values.
    * **mem_usage_avg**
    * **mem_usage_max**
* **gpu_usage**
    * Trickier, do we want the total sum or average of all nodes?
    * **gpu_usage_avg**
* **gpfs_read**
    * I think the average across all nodes should be appropriate to get a sense for GPFS usage in general.
    * **gpfs_read_avg**
* **gpfs_write**
    * I think the average across all nodes should be appropriate to get a sense for GPFS usage in general.
    * **gpfs_write_avg**
* **allocation_id**
    * Pivot value, summations and math operations pivot on this data point.
* **cpu_usage**
    * Average should be fine, can show trends. If combined with median, could indicate uneven workload.
    * **cpu_usage_avg**
    * **cpu_usage_median**
* **ib_tx**
    * **ib_tx_avg** 
* **ib_rx**
    * **ib_rx_avg** 
* **energy**
    * **energy_avg**
    * **energy_median**

In [7]:
sc.setLogLevel("DEBUG")
# Let's Write a Query!

NHistQuery = '''{ "query" :
    {
        "match_all": {"data.allocation_id"}
    },
    "aggs":{
        "histogram": {
            "field" : "data.allocation_id",
            "interval" : 1
        }
    },
    "size" : 0
}'''

es_conf = {"es.resource": "{0}".format(NODE_HIST),
          "es.nodes"    : NODES,
          "es.port"     : PORTS,
          "es.query"    : NHistQuery, }

nodeHist = sc.newAPIHadoopRDD("org.elasticsearch.hadoop.mr.EsInputFormat",\
                         "org.apache.hadoop.io.NullWritable", \
                         "org.elasticsearch.hadoop.mr.LinkedMapWritable",conf=es_conf)


nodeHist.first()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.newAPIHadoopRDD.
: java.lang.IllegalArgumentException: Failed to parse query: { "query" :
    {
        "match_all": {data.allocation_id"}
    },
    "aggs":{
        "histogram": {
            "field" : "data.allocation_id",
            "interval" : 1
        }
    },
    "size" : 0
}
	at org.elasticsearch.hadoop.rest.query.QueryUtils.parseQuery(QueryUtils.java:59)
	at org.elasticsearch.hadoop.rest.query.QueryUtils.parseQueryAndFilters(QueryUtils.java:81)
	at org.elasticsearch.hadoop.rest.RestService.findSlicePartitions(RestService.java:324)
	at org.elasticsearch.hadoop.rest.RestService.findPartitions(RestService.java:270)
	at org.elasticsearch.hadoop.mr.EsInputFormat.getSplits(EsInputFormat.java:414)
	at org.elasticsearch.hadoop.mr.EsInputFormat.getSplits(EsInputFormat.java:395)
	at org.apache.spark.rdd.NewHadoopRDD.getPartitions(NewHadoopRDD.scala:130)
	at org.apache.spark.rdd.RDD$$anonfun$partiti

In [13]:
#nodeHist.take(10)

In [ ]:
nodeHist.count()